# 任务 3：使用 Amazon Bedrock 进行问答

在此笔记本中，您将学习通过向 Bedrock Titan 模型发送包含完整相关上下文的请求并获得响应，从而使用该模型为查询提供信息响应，这可解决无需事先准备文档和编制索引即可通过模型为查询返回事实回答的难题。

此笔记本会模拟**检索增强生成 (RAG)** 的功能，但并未实际使用 RAG。此方法适用于简短文档或单个应用程序；它可能无法扩展到企业级问答场景，因为有些大型企业文档超出了允许发送给模型的提示词长度。

**问答 (QA)** 是一项重要的任务，涉及针对以自然语言提出的事实查询提取答案。通常，问答系统会根据包含结构化或非结构化数据的知识库来处理查询，并生成包含准确信息的响应。确保高准确率是开发实用、可靠且值得信赖的问答系统的关键所在，对于企业使用案例尤其如此。


## 场景

您尝试模拟以下场景：在 AnyCompany，您要求问答模型提供有关为他们制造的特定车辆型号更换轮胎的信息。您首先使用“零样本”方法向该模型进行查询，看看它能否仅根据其训练数据就提供相关的回答。

但是，您意识到该模型似乎在“产生幻觉”并给出了一般性的回答，当您提供一个虚假车辆型号时也会得到相似的响应就证明了这一点。这意味着需要通过示例公司的实际车辆手册来提供每种型号的轮胎细节，从而增强模型训练。

在本实验中，您将在没有外部数据的情况下模拟这种“检索增强生成”(RAG) 方法。您将提供详细的手册摘录，其中说明了如何为 AnyCompany 的 Z 型号车辆更换轮胎。您将测试模型现在能否利用此上下文中的示例内容提供自定义的准确回答。

## 任务 3.1：环境设置

在此任务中，您将设置环境。

In [ ]:
#ignore warnings and create a service client by name using the default session.
import json
import os
import sys
import warnings

import boto3
import botocore

warnings.filterwarnings('ignore')
module_path = ".."
sys.path.append(os.path.abspath(module_path))
bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))



## 任务 3.2：基于模型的知识进行问答
在此部分中，我们将尝试使用 Bedrock 服务提供的模型，基于它在训练阶段获取的知识来回答问题。

在此任务中，您将使用 Amazon Bedrock 客户端的 invoke_model() 方法。使用此方法所必需的参数是 modelId（代表 Amazon Bedrock 模型 ARN）和 body（即任务的提示词）。

body 提示词会根据您选择的基础模型提供程序而变化。您需要按照下面的详细介绍进行演练。

```json
{
   modelId= model_id,
   contentType= "application/json",
   accept= "application/json",
   body=body
}

```

您将尝试使用 Bedrock 服务提供的模型，基于在训练阶段获取的知识来回答问题。

In [ ]:
prompt_data = """You are an helpful assistant. Answer questions in a concise way. If you are unsure about the
answer say 'I am unsure'

Question: How can I fix a flat tire on my AnyCompany AC8?
Answer:"""
parameters = {
    "maxTokenCount":512,
    "stopSequences":[],
    "temperature":0,
    "topP":0.9
    }

## 任务 3.3：通过传递 JSON body 来调用模型以生成响应

In [ ]:
#model configuration
body = json.dumps({"inputText": prompt_data, "textGenerationConfig": parameters})
modelId = "amazon.titan-text-express-v1"  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"
try:
    
    response = bedrock_client.invoke_model(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    response_body = json.loads(response.get("body").read())
    answer = response_body.get("results")[0].get("outputText")
    print(answer.strip())

except botocore.exceptions.ClientError as error:
    if  error.response['Error']['Code'] == 'AccessDeniedException':
        print(f"\x1b[41m{error.response['Error']['Message']}\
        \nTo troubeshoot this issue please refer to the following resources.\
         \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
         \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")      
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error


该模型会为您提供回答，概述更换汽车漏气轮胎的过程，但同样的说明可能适用于任何汽车。遗憾的是，这不是适合 AnyCompany AC8 的正确答案，因为它没有备胎。之所以发生这种情况，是因为该模型已经基于包含汽车轮胎更换说明的数据进行了训练。

可反映该问题的另一个示例是，尝试为诸如 Amazon Tirana 这样完全虚假的汽车品牌和型号询问相同的问题。

In [ ]:
prompt_data = "How can I fix a flat tire on my Amazon Tirana?"
body = json.dumps({"inputText": prompt_data, 
                   "textGenerationConfig": parameters})
modelId = "amazon.titan-text-express-v1"  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"

response = bedrock_client.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())
answer = response_body.get("results")[0].get("outputText")
print(answer.strip())

对于提示词问题，该模型无法提供现实的回答。

要修复此问题，并让模型根据适合您的汽车型号的具体说明提供回答，您可以通过在提示词中提供附加知识库来即时增强模型的知识。

让我们看看如何使用该方法来改进应用程序。

假设以下是 AnyCompany AC8 手册的摘录（它实际上并不是真正的手册，但假设它是）。此文档也很简短易用，完全可以放入 Titan 大型模型的上下文窗口中。

```plain
Tires and Tire Pressure:

Tires are made of black rubber and are mounted on the wheels of your vehicle. They provide the necessary grip for driving, cornering, and braking. Two important factors to consider are tire pressure and tire wear, as they can affect the performance and handling of your car.

Where to Find Recommended Tire Pressure:

You can find the recommended tire pressure specifications on the inflation label located on the driver's side B-pillar of your vehicle. Alternatively, you can refer to your vehicle's manual for this information. The recommended tire pressure may vary depending on the speed and the number of occupants or maximum load in the vehicle.

Reinflating the Tires:

When checking tire pressure, it is important to do so when the tires are cold. This means allowing the vehicle to sit for at least three hours to ensure the tires are at the same temperature as the ambient temperature.

To reinflate the tires:

    Check the recommended tire pressure for your vehicle.
    Follow the instructions provided on the air pump and inflate the tire(s) to the correct pressure.
    In the center display of your vehicle, open the "Car status" app.
    Navigate to the "Tire pressure" tab.
    Press the "Calibrate pressure" option and confirm the action.
    Drive the car for a few minutes at a speed above 30 km/h to calibrate the tire pressure.

Note: In some cases, it may be necessary to drive for more than 15 minutes to clear any warning symbols or messages related to tire pressure. If the warnings persist, allow the tires to cool down and repeat the above steps.

Flat Tire:

If you encounter a flat tire while driving, you can temporarily seal the puncture and reinflate the tire using a tire mobility kit. This kit is typically stored under the lining of the luggage area in your vehicle.

Instructions for Using the Tire Mobility Kit:

    Open the tailgate or trunk of your vehicle.
    Lift up the lining of the luggage area to access the tire mobility kit.
    Follow the instructions provided with the tire mobility kit to seal the puncture in the tire.
    After using the kit, make sure to securely put it back in its original location.
    Contact Rivesla or an appropriate service for assistance with disposing of and replacing the used sealant bottle.

Please note that the tire mobility kit is a temporary solution and is designed to allow you to drive for a maximum of 10 minutes or 8 km (whichever comes first) at a maximum speed of 80 km/h. It is advisable to replace the punctured tire or have it repaired by a professional as soon as possible.
```

In [ ]:
context = """Tires and tire pressure:

Tires are made of black rubber and are mounted on the wheels of your vehicle. They provide the necessary grip for driving, cornering, and braking. Two important factors to consider are tire pressure and tire wear, as they can affect the performance and handling of your car.

Where to find recommended tire pressure:

You can find the recommended tire pressure specifications on the inflation label located on the driver's side B-pillar of your vehicle. Alternatively, you can refer to your vehicle's manual for this information. The recommended tire pressure may vary depending on the speed and the number of occupants or maximum load in the vehicle.

Reinflating the tires:

When checking tire pressure, it is important to do so when the tires are cold. This means allowing the vehicle to sit for at least three hours to ensure the tires are at the same temperature as the ambient temperature.

To reinflate the tires:

    Check the recommended tire pressure for your vehicle.
    Follow the instructions provided on the air pump and inflate the tire(s) to the correct pressure.
    In the center display of your vehicle, open the "Car status" app.
    Navigate to the "Tire pressure" tab.
    Press the "Calibrate pressure" option and confirm the action.
    Drive the car for a few minutes at a speed above 30 km/h to calibrate the tire pressure.

Note: In some cases, it may be necessary to drive for more than 15 minutes to clear any warning symbols or messages related to tire pressure. If the warnings persist, allow the tires to cool down and repeat the above steps.

Flat Tire:

If you encounter a flat tire while driving, you can temporarily seal the puncture and reinflate the tire using a tire mobility kit. This kit is typically stored under the lining of the luggage area in your vehicle.

Instructions for using the tire mobility kit:

    Open the tailgate or trunk of your vehicle.
    Lift up the lining of the luggage area to access the tire mobility kit.
    Follow the instructions provided with the tire mobility kit to seal the puncture in the tire.
    After using the kit, make sure to securely put it back in its original location.
    Contact AnyCompany or an appropriate service for assistance with disposing of and replacing the used sealant bottle.

Please note that the tire mobility kit is a temporary solution and is designed to allow you to drive for a maximum of 10 minutes or 8 km (whichever comes first) at a maximum speed of 80 km/h. It is advisable to replace the punctured tire or have it repaired by a professional as soon as possible."""

##### 现在，将整个摘录连同问题一起传递给模型。

In [ ]:
question = "How can I fix a flat tire on my AnyCompany AC8?"
prompt_data = f"""Answer the question based only on the information provided between ## and give step by step guide.
#
{context}
#

Question: {question}
Answer:"""

### 任务 3.4：通过 boto3 调用模型以生成响应

In [ ]:
body = json.dumps({"inputText": prompt_data, "textGenerationConfig": parameters})
modelId = "amazon.titan-text-express-v1"  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"

response = bedrock_client.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())
answer = response_body.get("results")[0].get("outputText")
print(answer.strip())

由于模型需要一些时间来理解上下文并为您生成相关的回答，这可能会导致用户体验不佳，因为用户必须等待几秒钟才能获得响应。

Bedrock 还支持流式传输功能，可以让服务在模型生成 Token 时生成输出。以下示例展示了如何实现这一点。

In [ ]:
from IPython.display import display_markdown,Markdown,clear_output

In [ ]:
# response with stream
response = bedrock_client.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
stream = response.get('body')
output = []
i = 1
if stream:
    for event in stream:
        chunk = event.get('chunk')
        if chunk:
            chunk_obj = json.loads(chunk.get('bytes').decode())
            text = chunk_obj['outputText']
            clear_output(wait=True)
            output.append(text)
            display_markdown(Markdown(''.join(output)))
            i+=1

响应提供了有关如何更换轮胎的概括性分步说明。

现在，您已经学习了如何利用检索增强生成 (RAG) 或增强过程来根据提供的具体上下文和信息生成精心编排的响应。

### 自行尝试
- 将提示词更改为您的特定使用案例，并评估不同模型的输出。
- 尝试不同的 Token 长度，了解服务的延迟和响应能力。
- 应用不同的提示词工程原则，获得更好的输出。

### 清理

您已完成此笔记本。要进入本实验的下一部分，请执行以下操作：

- Close this notebook file and continue with **Task 4**.